# WEEKLY MARKUP ANALYSIS

In [1140]:
#Paqueterias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

pd.options.display.float_format = '{:,.2f}'.format
input_path='/Users/ivanjardon/Downloads/pulsemarkupbystore/'
output_path='/Users/ivanjardon/Documents/'

In [1141]:
#Leer datos
markup_mx=pd.read_excel(input_path+'markup_mx.xlsx')
markup_cl=pd.read_excel(input_path+'markup_cl.xlsx')
markup_br=pd.read_excel(input_path+'markup_br.xlsx')
markup_ca=pd.read_excel(input_path+'markup_ca.xlsx')
markup_co=pd.read_excel(input_path+'markup_co.xlsx')
markup_pe=pd.read_excel(input_path+'markup_pe.xlsx')
markup_cr=pd.read_excel(input_path+'markup_cr.xlsx')
markup_us=pd.read_excel(input_path+'markup_us.xlsx')

In [1142]:
#Append
markup_cs = pd.concat([markup_mx,markup_cl,markup_br,markup_ca,markup_co,markup_pe,markup_cr,markup_us],
                               ignore_index=True)
#Add column markup_type
markup_cs['markup_type'] = np.where(markup_cs['markup_pctg']>1, 'markup', np.where(markup_cs['markup_pctg']<-1,'markdown','no markup'))
#Add column service type
markup_cs['service_type'] = np.where(markup_cs['is_longtail']==False, 'Full service','Self service')
#Add column markup by order
markup_cs['markup_by_order_usd'] = markup_cs['markup_usd']/markup_cs['orders']
#Add column exhange_rate
markup_cs['exchange_rate'] = markup_cs['total_receipt_local']/markup_cs['total_receipt_usd']
#set_index
markup_cs.set_index('store_id')

#extract week
week=markup_cs['week'].unique()

                               
display(markup_cs.head())
markup_cs.to_excel(output_path+'markup_by_store_'+str(week[0])+'.xlsx')

,week,country,is_longtail,store_id,store,orders,products_charged_local,total_receipt_local,markup_local,products_charged_usd,total_receipt_usd,markup_usd,markup_pctg,markup_type,service_type,markup_by_order_usd,exchange_rate
0,2021-05,MX,True,59,Carnes Ramos,901,"603,990.85","612,177.63","-8,186.78","30,059.54","30,466.98",-407.44,-1.34,markdown,Self service,-0.45,20.09
1,2021-05,MX,True,5177,La Castellana,17,"20,605.28","23,335.65","-2,730.37","1,025.49","1,161.37",-135.89,-11.70,markdown,Self service,-7.99,20.09
2,2021-05,MX,False,4785,City Market Farmacia,96,"67,205.95","68,817.83","-1,611.88","3,344.72","3,424.94",-80.22,-2.34,markdown,Full service,-0.84,20.09
3,2021-05,MX,True,490,Catabocata,13,"12,796.00","14,252.00","-1,456.00",636.83,709.30,-72.46,-10.22,markdown,Self service,-5.57,20.09
4,2021-05,MX,False,9,City Market,10740,"13,804,293.31","13,805,520.65","-1,227.34","687,014.86","687,075.94",-61.08,-0.01,no markup,Full service,-0.01,20.09


In [1143]:
#Cosulta tienda
markup_cs.query('store_id=="4"')

,week,country,is_longtail,store_id,store,orders,products_charged_local,total_receipt_local,markup_local,products_charged_usd,total_receipt_usd,markup_usd,markup_pctg,markup_type,service_type,markup_by_order_usd,exchange_rate
1406,2021-05,CL,False,4,Jumbo,71657,"4,982,847,916.74","4,799,039,657.00","183,808,259.74","6,795,090.35","6,544,431.74","250,658.61",3.83,markup,Full service,3.50,733.30


In [1144]:
#MARKUP TOTAL CS
markup_total_cs=markup_cs.groupby('week').agg({'products_charged_usd':'sum','total_receipt_usd':'sum','markup_usd':'sum','orders':'sum'})
markup_total_cs['markup_pctg']=markup_total_cs['markup_usd']*100/markup_total_cs['total_receipt_usd']
display(markup_total_cs[['orders','markup_usd','markup_pctg']])


,orders,markup_usd,markup_pctg
week,,,
2021-05,366299,"557,120.86",2.53


In [1145]:
#MARKUP BY COUNTRY
markup_by_country=markup_cs.groupby('country').agg({'products_charged_local':'sum','total_receipt_local':'sum','markup_local':'sum','markup_usd':'sum','orders':'sum'})
markup_by_country['markup_pctg']=markup_by_country['markup_local']*100/markup_by_country['total_receipt_local']
display(markup_by_country[['orders','markup_local','markup_usd','markup_pctg']].style.background_gradient(cmap='RdBu',subset=['markup_pctg']).format("{:,.2f}"))

#Exportar a excel
markup_by_country.to_excel(output_path+'markup_by_country_'+str(week[0])+'.xlsx')

,orders,markup_local,markup_usd,markup_pctg
country,,,,
BR,"47,621.00","77,381.32","14,276.30",0.71
CA,"36,345.00","46,460.13","36,445.58",1.72
CL,"136,875.00","215,421,902.39","293,770.01",3.08
CO,"3,825.00","27,374,233.87","7,671.83",3.14
CR,749.00,"268,332.06",438.25,1.45
MX,"117,940.00","3,406,355.26","169,528.18",2.47
PE,"14,945.00","97,964.19","26,898.46",3.04
US,"7,999.00","8,092.26","8,092.26",2.14


In [1146]:
#MARKUP BY SERVICE TYPE
markup_by_service_type=markup_cs.groupby(['country','is_longtail']).agg({'products_charged_local':'sum','total_receipt_local':'sum','markup_local':'sum','markup_usd':'sum','orders':'sum'})
markup_by_service_type['markup_pctg']=markup_by_service_type['markup_local']*100/markup_by_service_type['total_receipt_local']
display(markup_by_service_type[['orders','markup_local','markup_usd','markup_pctg']].style.background_gradient(cmap='RdBu',subset=['markup_pctg']).format("{:,.2f}"))

#Exportar a excel
markup_by_service_type.to_excel(output_path+'markup_by_service_type_'+str(week[0])+'.xlsx')

In [1147]:
#MARKUP TOTAL CS by service type
markup_total_cs=markup_cs.groupby(['week','service_type']).agg({'products_charged_usd':'sum','total_receipt_usd':'sum','markup_usd':'sum','orders':'sum'})
markup_total_cs['markup_pctg']=markup_total_cs['markup_usd']*100/markup_total_cs['total_receipt_usd']
display(markup_total_cs[['orders','markup_usd','markup_pctg']])


orders  markup_usd  markup_pctg
week    service_type                                 
2021-05 Full service  318941  556,294.30         2.73
        Self service   47358      826.56         0.05

In [1148]:
#TOP OFFENDERS MARKDOWN
top_offenders=markup_cs.query('markup_pctg<-1').sort_values(by='markup_usd').iloc[0:10,:]
display(top_offenders[['week','country','store','service_type','orders','markup_local','markup_usd','markup_pctg']].style.background_gradient(cmap='Reds_r',subset=['markup_pctg']).format("{:,.2f}",subset=['markup_local','markup_usd','markup_pctg']))
#Exportar a excel
top_offenders.to_excel(output_path+'top_offenders_markdown_'+str(week[0])+'.xlsx')

,week,country,store,service_type,orders,markup_local,markup_usd,markup_pctg
1736,2021-05,CA,Organic Garage,Full service,815,-979.40,-768.29,-1.30
1407,2021-05,BR,Cobasi,Self service,411,"-3,785.49",-698.40,-6.21
1989,2021-05,US,Gristedes,Full service,356,-581.20,-581.20,-2.56
0,2021-05,MX,Carnes Ramos,Self service,901,"-8,186.78",-407.44,-1.34
1990,2021-05,US,Sarah's Tent,Self service,9,-267.05,-267.05,-48.35
1408,2021-05,BR,Hippo,Full service,29,"-1,320.22",-243.57,-16.21
1410,2021-05,BR,Akkí Atacadista,Full service,67,"-1,027.28",-189.53,-6.09
1411,2021-05,BR,Supermercado Tatico,Full service,168,-771.00,-142.24,-2.09
1,2021-05,MX,La Castellana,Self service,17,"-2,730.37",-135.89,-11.70
457,2021-05,CL,Mil sabores,Self service,47,"-86,756.00",-118.31,-5.33


In [1149]:
#TOP OFFENDERS MARKUP
top_offenders=markup_cs.query('markup_pctg>1').sort_values(by='markup_usd',ascending=False).iloc[0:10,:]
display(top_offenders[['week','country','store','service_type','orders','markup_local','markup_usd','markup_pctg']].style.background_gradient(cmap='Blues',subset=['markup_pctg']).format("{:,.2f}",subset=['markup_local','markup_usd','markup_pctg']))

#Exportar a excel
top_offenders.to_excel(output_path+'top_offenders_markup_'+str(week[0])+'.xlsx')

,week,country,store,service_type,orders,markup_local,markup_usd,markup_pctg
1406,2021-05,CL,Jumbo,Full service,71657,"183,808,259.74","250,658.61",3.83
456,2021-05,MX,HEB,Full service,19500,"1,945,757.48","96,836.85",7.65
455,2021-05,MX,Chedraui,Full service,29349,"636,660.49","31,685.45",1.81
1801,2021-05,CA,Metro,Full service,11875,"27,453.37","21,535.76",3.07
1405,2021-05,CL,Lider,Full service,10926,"12,829,880.40","17,496.06",2.32
1964,2021-05,PE,Wong,Full service,7092,"57,422.63","15,766.79",3.27
454,2021-05,MX,Costco,Full service,15973,"315,666.09","15,710.13",1.27
453,2021-05,MX,Soriana,Full service,11187,"313,081.75","15,581.52",2.90
1735,2021-05,BR,Carrefour Hiper,Full service,13394,"42,117.17","7,770.32",1.37
1799,2021-05,CA,Longo's,Full service,2078,"7,149.43","5,608.36",4.55


In [1150]:
#Markup types store count
markup_cs_type=markup_cs.groupby(['country','service_type','markup_type']).agg({'store':'count'}).reset_index()
markup_cs_type=markup_cs_type.rename(columns = {'store': 'count'})

#Pivot
markup_cs_type_pivot = markup_cs_type.pivot_table(values="count",index=['country','service_type'],columns=["markup_type"],aggfunc=np.sum,fill_value=0)
display(markup_cs_type_pivot.style.background_gradient('Greys'))



In [1151]:
#Gráfico "Stores markup kind by country and service type"

markup_cs_type['percentage']=markup_cs.groupby(['country', 'service_type','markup_type']).size().groupby(['country', 'service_type']).apply(lambda x: 100 * x / float(x.sum())).values.round(0)

fig = px.bar(markup_cs_type, 
             x="country",
             y='percentage', 
             color='markup_type',
             hover_data=['count'],
             color_discrete_sequence=['green','blue','red'],range_y=[0,100],
             title="Stores markup kind by country and service type, week: "+str(week[0]),
             labels={'count':'Number of Stores','country':'Country','service_type':'Type','markup_type':'Markup type','percentage':'Percentage %'}
             ,facet_row="service_type",
             category_orders={"service_type": ["Full service", "Self service"],'markup_type':['no markup','markup','markdown']},
            text='percentage')
fig.update_yaxes(matches=None)
#fig.add_annotation(x='CO', y=1,
           # text="Markup/markdown considers relative markup above 1% or below -1%",showarrow=False)
fig.show()


In [1152]:
#Gráfico"Markup percentage distributions by country and service type"
fig = px.box(markup_cs.query('orders>7'), x="country", y="markup_pctg", 
             color="service_type",hover_data=['markup_usd','orders','markup_by_order_usd'],hover_name='store',range_y=[-8,12]
             ,title="Markup percentage distributions by country and service type, week: "+str(week[0]),
             labels={'markup_pctg':'Markup percentage (%)','country':'Country','service_type':'Service type','markup_usd':'Markup (USD)','orders':'Total orders','markup_by_order_usd':'Markup per order (USD)'}
            )

fig.update_yaxes(dtick=2)
fig.add_annotation(x='CR', y=-6.5,
            text="Data from stores with at least 8 weekly orders",showarrow=False)
          
fig.show()

In [1153]:
#Gráfico"Markup percentage distributions by country and service type"
fig = px.box(markup_cs.query('orders>7'), x="country", y="markup_by_order_usd", 
             color="service_type",hover_data=['markup_usd','orders'],hover_name='store',range_y=[-5,5]
             ,title="Markup per order distributions by country and service type, week: "+str(week[0]),
             labels={'markup_by_order_usd':'Markup per order (USD)','markup_usd':'Markup (USD)','country':'Country','service_type':'Service type','orders':'Total orders'}
            )

fig.update_yaxes(dtick=1)

fig.add_annotation(x='CR', y=-3.5,
            text="Data from stores with at least 8 weekly orders",showarrow=False)
          
fig.show()

In [1227]:
#Gráfico
top_grossing_stores=markup_cs.sort_values(by='orders',ascending=False).iloc[0:100,:]

fig = px.scatter(top_grossing_stores, 
                 y="markup_pctg", x="markup_by_order_usd",color='country',hover_data=['markup_usd','orders']
                 ,hover_name='store',opacity=0.6,title="Top 100 grossing stores markup percentage vs markup per order, week: "+str(week[0]),size='orders',size_max=50
                ,labels={'markup_by_order_usd':'Markup per order (USD)','markup_usd':'Markup (USD)','country':'Country','service_type':'Service type','orders':'Total orders','markup_pctg':'Markup percentage','avg_ticket':'Average ticket'}
                )

fig.update_traces(textposition='top center')
fig.update_yaxes(dtick=2)


coordenada=top_grossing_stores[['markup_by_order_usd','markup_pctg','store','country']].iloc[0:5,:].values.tolist()
    
for x,y in enumerate(coordenada):
    fig.add_annotation(x=coordenada[x][0], y=coordenada[x][1],
    text=str(coordenada[x][2])+' '+str(coordenada[x][3]),showarrow=False,arrowhead=1)


fig.add_annotation(x=4.5, y=-4,
           text="Bubble size=orders",showarrow=False)



fig.show()
top_grossing_stores.head()                                                              

,week,country,is_longtail,store_id,store,orders,products_charged_local,total_receipt_local,markup_local,products_charged_usd,total_receipt_usd,markup_usd,markup_pctg,markup_type,service_type,markup_by_order_usd,exchange_rate
1406,2021-05,CL,False,4,Jumbo,71657,"4,982,847,916.74","4,799,039,657.00","183,808,259.74","6,795,090.35","6,544,431.74","250,658.61",3.83,markup,Full service,3.50,733.30
455,2021-05,MX,False,22,Chedraui,29349,"35,879,972.53","35,243,312.04","636,660.49","1,785,681.72","1,753,996.28","31,685.45",1.81,markup,Full service,1.08,20.09
456,2021-05,MX,False,25,HEB,19500,"27,393,940.01","25,448,182.53","1,945,757.48","1,363,347.14","1,266,510.29","96,836.85",7.65,markup,Full service,4.97,20.09
1800,2021-05,CA,False,537,Walmart,18115,"1,342,474.33","1,333,021.67","9,452.66","1,053,101.93","1,045,686.80","7,415.13",0.71,no markup,Full service,0.41,1.27
454,2021-05,MX,False,7,Costco,15973,"25,191,336.08","24,875,669.99","315,666.09","1,253,727.51","1,238,017.37","15,710.13",1.27,markup,Full service,0.98,20.09
